In [6]:
## Testing association
import pandas as pd
import numpy as np
df = pd.read_csv('datasets/WEB_LOG_DATA.csv', index_col=0)

df.head()

,date_time,request,step,session,user_id
ip,,,,,
c210-49-32-6.rochd2.,18/Aug/2017:21:25:07,/,1,3,3
visp.inabox.telstra.,19/Aug/2017:08:24:28,/,1,12,12
adsl-61-95-54-84.requ,19/Aug/2017:08:33:01,/,1,13,13
d220-236-91-52.adsl.n,19/Aug/2017:09:16:06,/,1,15,15
allptrs.eq.edu.au,19/Aug/2017:09:47:54,/,1,22,22


In [16]:
df.info()
df2 = df[df['request'] != '/robots.txt']
df2 = df2[df2['request'] != '/favicon.ico']
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5866 entries, c210-49-32-6.rochd2. to lj2362.inktomisearch
Data columns (total 5 columns):
date_time    5866 non-null object
request      5866 non-null object
step         5866 non-null int64
session      5866 non-null int64
user_id      5866 non-null int64
dtypes: int64(3), object(2)
memory usage: 275.0+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 4917 entries, c210-49-32-6.rochd2. to lj2362.inktomisearch
Data columns (total 5 columns):
date_time    4917 non-null object
request      4917 non-null object
step         4917 non-null int64
session      4917 non-null int64
user_id      4917 non-null int64
dtypes: int64(3), object(2)
memory usage: 230.5+ KB


In [62]:
transactions = df2.groupby(['user_id'])['request'].apply(list)

from apyori import apriori
# type cast the transactions from pandas into normal list format and run apriori
transaction_list = list(transactions)
results = list(apriori(transaction_list, min_support=0.035, min_confidence=0.1))
# print first 5 rules
print(results[:5])

[RelationRecord(items=frozenset({'/'}), support=0.4399271844660194, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'/'}), confidence=0.4399271844660194, lift=1.0)]), RelationRecord(items=frozenset({'/newfarm/'}), support=0.15351941747572814, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'/newfarm/'}), confidence=0.15351941747572814, lift=1.0)]), RelationRecord(items=frozenset({'/newfarm/javascript/menu.js'}), support=0.16868932038834952, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'/newfarm/javascript/menu.js'}), confidence=0.16868932038834952, lift=1.0)]), RelationRecord(items=frozenset({'/newfarm/pricelist'}), support=0.10800970873786407, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'/newfarm/pricelist'}), confidence=0.10800970873786407, lift=1.0)]), RelationRecord(items=frozenset({'/newfarm/pricelist/'}), support=0.1049757281553398, ordere

In [63]:
def convert_apriori_results_to_pandas_df(results):
    rules = []

    for rule_set in results:
        for rule in rule_set.ordered_statistics:
            # items_base = left side of rules, items_add = right side
            # support, confidence and lift for respective rules
            rules.append([','.join(rule.items_base), ','.join(rule.items_add), rule_set.support, rule.confidence, rule.lift])

    # typecast it to pandas df
    return pd.DataFrame(rules, columns=['Left_side', 'Right_side', 'Support', 'Confidence', 'Lift'])

result_df = convert_apriori_results_to_pandas_df(results)

In [65]:
# sort all acquired rules descending by lift
print('v------------------------------Lift------------------------------------v')
result_df = result_df.sort_values(by='Lift', ascending=False)
result_df

v------------------------------Lift------------------------------------v


,Left_side,Right_side,Support,Confidence,Lift
57,/richlands/,/richlands,0.043083,0.910256,18.988640
56,/richlands,/richlands/,0.043083,0.898734,18.988640
109,"/newfarm/specials/,/newfarm/javascript/menu.js",/newfarm/specials,0.035194,0.753247,18.808343
59,/richlands/javascript/menu.js,/richlands/,0.036408,0.869565,18.372352
58,/richlands/,/richlands/javascript/menu.js,0.036408,0.769231,18.372352
64,"/newfarm/,/",/newfarm,0.035194,0.865672,16.983653
108,"/newfarm/specials,/newfarm/javascript/menu.js",/newfarm/specials/,0.035194,0.983051,16.701730
54,/newfarm/specials,/newfarm/specials/,0.038228,0.954545,16.217432
55,/newfarm/specials/,/newfarm/specials,0.038228,0.649485,16.217432
31,/newfarm/contact/,/newfarm/contact,0.043083,0.797753,14.939734
